<a href="https://colab.research.google.com/github/Ssactl/AH2179_Applied-AI-in-Transportation/blob/main/Module5_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#load the data

import pandas as pd
import numpy as np
#upload the dataset by downloading both datasets from canvas and upload it on colab

data_df = pd.read_csv("/dataset_exercise_5_clustering_highway_traffic.csv",sep=";")
# Read the evaluation dataset from a CSV file
data_eval_df = pd.read_csv("/evaluation_dataset_exercise_5_clustering_highway_traffic.csv", sep=";")

data_df

,PORTAL,Date,time_from,time_to,Interval_5,SPEED_MS_AVG,flow
0,"E4S 56,780",20210101,00:00:00,00:05:00,0,18.56,39
1,"E4S 56,780",20210101,00:05:00,00:10:00,1,20.39,18
2,"E4S 56,780",20210101,00:10:00,00:15:00,2,19.27,26
3,"E4S 56,780",20210101,00:15:00,00:20:00,3,19.52,52
4,"E4S 56,780",20210101,00:20:00,00:25:00,4,20.52,52
...,...,...,...,...,...,...,...
104838,"E4S 56,780",20211231,23:35:00,23:40:00,283,19.58,115
104839,"E4S 56,780",20211231,23:40:00,23:45:00,284,19.47,87
104840,"E4S 56,780",20211231,23:45:00,23:50:00,285,19.77,130
104841,"E4S 56,780",20211231,23:50:00,23:55:00,286,18.79,129


In [11]:
#day clustering

# Define the total number of 5-minute intervals in a day
nintvals = 288

def day_clustring(data_df):

    # Sort the DataFrame 'data_df' by columns "Date" and "Interval_5"
    data_df.sort_values(["Date", "Interval_5"])

    # Extract unique dates from the sorted DataFrame
    days = np.unique(data_df[['Date']].values.ravel())
    # Calculate the total number of unique days
    ndays = len(days)

    # Group the DataFrame 'data_df' by the "Date" column
    day_subsets_df = data_df.groupby(["Date"])

    # Create a matrix 'vectorized_day_dataset' filled with NaN values
    vectorized_day_dataset = np.zeros((ndays, nintvals))
    vectorized_day_dataset.fill(np.nan)

    # Loop through each unique day
    for i in range(0, ndays):
        # Get the DataFrame corresponding to the current day
        df_t = day_subsets_df.get_group(days[i])

        # Loop through each row in the current day's DataFrame
        for j in range(len(df_t)):
            # Get the current day's DataFrame
            df_t = day_subsets_df.get_group(days[i])

            # Extract the "Interval_5" and "flow" values and populate 'vectorized_day_dataset'
            vectorized_day_dataset[i, df_t.iloc[j]["Interval_5"]] = df_t.iloc[j]["flow"]

    # Print the resulting 'vectorized_day_dataset'
    return vectorized_day_dataset

vectorized_day_dataset=day_clustring(data_df)
vectorized_day_dataset_eval=day_clustring(data_eval_df)

print(vectorized_day_dataset)
print(vectorized_day_dataset_eval)


[[ 39.  18.  26. ...  32.  39.  34.]
 [ 30.  32.  27. ...  44.  41.  39.]
 [ 36.  44.  52. ...  50.  45.  23.]
 ...
 [ 20.  34.  31. ...  38.  42.  36.]
 [ 36.  40.  25. ...  38.  56.  35.]
 [ 33.  32.  34. ... 130. 129. 117.]]
[[16.74 17.57 16.94 ... 19.43 19.88 19.73]
 [19.17 19.26 20.98 ... 19.34 21.22 19.79]
 [19.69 19.39 19.76 ... 20.51 20.5  19.6 ]
 ...
 [19.65 22.59 20.8  ... 19.23 20.32 20.18]
 [19.75 19.96 20.95 ... 17.95 19.14 19.55]
 [20.82 20.28 20.5  ... 19.72 20.64 20.23]]


In [12]:
#remove missing values

nans_per_day = np.sum(np.isnan(vectorized_day_dataset),1)
days = np.unique(data_df[['Date']].values.ravel())
vectorized_day_dataset_no_nans = vectorized_day_dataset[np.where(nans_per_day == 0)[0],:]
days_not_nans = days[np.where(nans_per_day == 0)[0]]

nans_per_day_eval = np.sum(np.isnan(vectorized_day_dataset_eval), 1)
days_eval = np.unique(data_eval_df[['Date']].values.ravel())
vectorized_day_dataset_no_nans_eval = vectorized_day_dataset_eval[np.where(nans_per_day_eval == 0)[0], :]
days_not_nans_eval = days_eval[np.where(nans_per_day_eval == 0)[0]]
# Calculate the total number of days in the filtered evaluation dataset
ndays_eval_not_nans = len(days_not_nans_eval)


In [6]:
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import calinski_harabasz_score, silhouette_score, davies_bouldin_score
import sklearn.metrics.pairwise as dis_lib

In [41]:
#clustering evaluation

#internal evaluation
def internal_eval(vectorized_day_dataset_no_nans,cluster_labels):
    # Calculate the Silhouette Score
    SC_score = silhouette_score(vectorized_day_dataset_no_nans, cluster_labels)
    # Silhouette Score measures the quality of clusters, higher values indicate better separation.

    # Calculate the Davies-Bouldin Score
    DB_score = davies_bouldin_score(vectorized_day_dataset_no_nans, cluster_labels)
    # Davies-Bouldin Score measures the average similarity between each cluster and its most similar cluster, lower values indicate better separation.

    # Calculate the Calinski-Harabasz Score
    CH_score = calinski_harabasz_score(vectorized_day_dataset_no_nans, cluster_labels)
    # Calinski-Harabasz Score measures the ratio of between-cluster variance to within-cluster variance, higher values indicate better separation.

    # Print the computed cluster quality scores
    print('Silhouette Score:', SC_score)
    print('Davies-Bouldin Score:', DB_score)
    print('Calinski-Harabasz Score:', CH_score)



#external evaluation with short-term prediction
# Define a function to find the closest centroid to a new data point within a specified day-time interval range
def find_the_closest_centroid(centroids, new_day, from_interval: int, to_interval: int):
    closest_centroid = None
    closest_dist = None

    # Iterate through each centroid
    for i in range(0, len(centroids)):
        # Calculate the Euclidean distance between the centroid and the new data point
        ed_t = dis_lib.paired_distances(centroids[i], new_day, metric='euclidean')

        # Check if the current centroid is closer than the previously closest one
        if closest_centroid is None or closest_dist > ed_t:
            closest_centroid = i
            closest_dist = ed_t

    return closest_centroid



def external_eval(n_clusters_t,nintvals,ndays_eval_not_nans,cluster_labels):
    # Initialize a list to store centroid data
    centroids = []

    # Calculate centroids for each cluster
    for i in np.unique(cluster_labels):
        centroid = np.nanmean(vectorized_day_dataset_no_nans[np.where(cluster_labels == i)[0], :], 0).reshape(1, nintvals)
        centroids.append(centroid)

    # Define the number of past intervals to consider for classification
    n_past_intervals_for_classification = 5

    # Initialize variables to calculate accuracy metrics
    total_mae = 0
    total_mape = 0
    prediction_counts = 0

    # Loop through each day in the evaluation dataset with no missing values
    for i in range(0, ndays_eval_not_nans):
        # Loop through intervals from n_past_intervals_for_classification to nintvals - 1
        for j in range(n_past_intervals_for_classification, nintvals - 1):
            # Find the closest centroid for the current data point
            centroid_index = find_the_closest_centroid(centroids, vectorized_day_dataset_no_nans_eval[i].reshape(1, nintvals), j - n_past_intervals_for_classification, j)

            # Predict the value for the next interval
            predicted_value = centroids[centroid_index][0, j + 1]

            # Calculate Mean Absolute Error (MAE) and Mean Absolute Percentage Error (MAPE)
            mae_t = abs(predicted_value - vectorized_day_dataset_no_nans_eval[i][j + 1])
            mape_t = abs(predicted_value - vectorized_day_dataset_no_nans_eval[i][j + 1]) / float(vectorized_day_dataset_no_nans_eval[i][j + 1])

            # Accumulate MAE, MAPE, and count of predictions
            total_mae += mae_t
            total_mape += mape_t
            prediction_counts += 1

    # Calculate and print the prediction accuracy metrics
    print('Prediction accuracy MAE:', total_mae / prediction_counts)
    print('Prediction accuracy MAPE:', total_mape / prediction_counts)

In [13]:
#clustering
from sklearn.model_selection import GridSearchCV

In [19]:
#k-means

n_clusters=range(2,11)
clusters=None
cluster_labels=None

for i in n_clusters:
  clusters = KMeans(n_clusters=i, random_state=0, n_init="auto").fit(vectorized_day_dataset_no_nans)

  print('=========')
  print('n_clusters:',i,'n_init: auto')
  print('=========')
  if clusters is not None:
    cluster_labels = clusters.labels_
  # Calculate the number of clusters by finding unique values in 'cluster_labels'
  n_clusters_t = len(np.unique(cluster_labels))
  internal_eval(vectorized_day_dataset_no_nans,cluster_labels)
  external_eval(n_clusters_t,nintvals,ndays_eval_not_nans)

n_clusters: 2 n_init: auto
Silhouette Score: 0.30762127308637705
Davies-Bouldin Score: 1.2677249598188514
Calinski-Harabasz Score: 174.1048835493247
Prediction accuracy MAE: 151.2845319079707
Prediction accuracy MAPE: 9.757155713658154
n_clusters: 3 n_init: auto
Silhouette Score: 0.3005356252450667
Davies-Bouldin Score: 1.19297485489169
Calinski-Harabasz Score: 115.20096139838287
Prediction accuracy MAE: 80.29886593346282
Prediction accuracy MAPE: 5.018853834616337
n_clusters: 4 n_init: auto
Silhouette Score: 0.27969101229629845
Davies-Bouldin Score: 1.1572414386234318
Calinski-Harabasz Score: 131.9873210744639
Prediction accuracy MAE: 68.64261075136191
Prediction accuracy MAPE: 3.9871097637946624
n_clusters: 5 n_init: auto
Silhouette Score: 0.2234369983793478
Davies-Bouldin Score: 1.4951233484702366
Calinski-Harabasz Score: 114.76424209725394
Prediction accuracy MAE: 59.98865522321583
Prediction accuracy MAPE: 3.318357803607238
n_clusters: 6 n_init: auto
Silhouette Score: 0.2368514222

In [27]:
#AgglomerativeClustering

n_clusters=range(2,11)
metric=['euclidean','manhattan','cosine']
linkage=['ward','complete','average','single']
clusters=None
cluster_labels=None

for i in n_clusters:
  for j in metric:
    for k in linkage:
      if (j!='euclidean') and k=='ward':
        continue
      clusters = AgglomerativeClustering(n_clusters=i,metric=j,linkage=k).fit(vectorized_day_dataset_no_nans)

      print('=========')
      print('n_clusters:',i,'metric:',j,'linkage:',k)
      print('=========')
      if clusters is not None:
        cluster_labels = clusters.labels_
      # Calculate the number of clusters by finding unique values in 'cluster_labels'
      n_clusters_t = len(np.unique(cluster_labels))
      internal_eval(vectorized_day_dataset_no_nans,cluster_labels)
      external_eval(n_clusters_t,nintvals,ndays_eval_not_nans)

n_clusters: 2 metric: euclidean linkage: ward
Silhouette Score: 0.29094541701496657
Davies-Bouldin Score: 1.279138038132245
Calinski-Harabasz Score: 159.8215045200973
Prediction accuracy MAE: 151.71073575720214
Prediction accuracy MAPE: 9.765341093379908
n_clusters: 2 metric: euclidean linkage: complete
Silhouette Score: 0.531750962730339
Davies-Bouldin Score: 0.5867453731064505
Calinski-Harabasz Score: 20.868050978169595
Prediction accuracy MAE: 59.98865522321583
Prediction accuracy MAPE: 3.318357803607238
n_clusters: 2 metric: euclidean linkage: average
Silhouette Score: 0.531750962730339
Davies-Bouldin Score: 0.5867453731064505
Calinski-Harabasz Score: 20.868050978169595
Prediction accuracy MAE: 59.98865522321583
Prediction accuracy MAPE: 3.318357803607238
n_clusters: 2 metric: euclidean linkage: single
Silhouette Score: 0.31675563294858017
Davies-Bouldin Score: 0.5667978189299843
Calinski-Harabasz Score: 2.7950676204068094
Prediction accuracy MAE: 148.70747558844778
Prediction accu

In [49]:
#DBSCAN

eps=[1000,1500]
min_samples=[2,3,4,5]
clusters=None
cluster_labels=None

for j in eps:
  for k in min_samples:
    clusters = DBSCAN(eps=j, min_samples = k).fit(vectorized_day_dataset_no_nans)

    print('=========')
    print('eps:',j,'min_samples:',k)
    print('=========')

    if clusters is not None:
      cluster_labels = clusters.labels_
    # print(np.unique(cluster_labels))
    # Calculate the number of clusters by finding unique values in 'cluster_labels'
    n_clusters_t = len(np.unique(cluster_labels))
    print('n_clusters:',n_clusters_t)
    internal_eval(vectorized_day_dataset_no_nans,cluster_labels)
    external_eval(n_clusters_t,nintvals,ndays_eval_not_nans,cluster_labels)

eps: 1000 min_samples: 2
n_clusters: 3
Silhouette Score: 0.2835750981116442
Davies-Bouldin Score: 2.2254060141342413
Calinski-Harabasz Score: 9.938030272461077
Prediction accuracy MAE: 63.58543735224614
Prediction accuracy MAPE: 3.5129508880355793
eps: 1000 min_samples: 3
n_clusters: 2
Silhouette Score: 0.34354697960895075
Davies-Bouldin Score: 2.436846439780033
Calinski-Harabasz Score: 11.023697758834578
Prediction accuracy MAE: 128.93331819583392
Prediction accuracy MAPE: 8.022345388158733
eps: 1000 min_samples: 4
n_clusters: 2
Silhouette Score: 0.34354697960895075
Davies-Bouldin Score: 2.436846439780033
Calinski-Harabasz Score: 11.023697758834578
Prediction accuracy MAE: 128.93331819583392
Prediction accuracy MAPE: 8.022345388158733
eps: 1000 min_samples: 5
n_clusters: 2
Silhouette Score: 0.34354697960895075
Davies-Bouldin Score: 2.436846439780033
Calinski-Harabasz Score: 11.023697758834578
Prediction accuracy MAE: 128.93331819583392
Prediction accuracy MAPE: 8.022345388158733
eps: 